In [3]:
import torch
import clip
from PIL import Image
import faiss
import numpy as np
import os


In [4]:
# Load CLIP
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)
print("device" ,device)

100%|███████████████████████████████████████| 338M/338M [00:14<00:00, 23.8MiB/s]


device cpu


In [6]:
# Encode dataset images
embeddings = []
ids = []

dataset_path = "dataset/"
for id_article in os.listdir(dataset_path):
    folder = os.path.join(dataset_path, id_article)
    if os.path.isdir(folder):
        for file in os.listdir(folder):
            image = preprocess(Image.open(os.path.join(folder, file))).unsqueeze(0).to(device)
            with torch.no_grad():
                emb = model.encode_image(image).cpu().numpy()
            embeddings.append(emb)
            ids.append(id_article)

embeddings = np.vstack(embeddings).astype("float32")


In [7]:
# Build FAISS index
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)


In [10]:
# Query with new image - Get top 3 most similar
query_img = preprocess(Image.open("test.jpeg")).unsqueeze(0).to(device)
with torch.no_grad():
    q_emb = model.encode_image(query_img).cpu().numpy()

# Search for top 3 most similar images
D, I = index.search(q_emb, 3)

print("Top 3 most similar articles:")
print("=" * 40)
for i in range(3):
    article_id = ids[I[0][i]]
    distance = D[0][i]
    # Convert L2 distance to similarity score (lower distance = higher similarity)
    # Using negative distance as similarity score (higher is better)
    similarity_score = -distance
    print(f"{i+1}. Article ID: {article_id}")
    print(f"   Similarity Score: {similarity_score:.4f}")
    print(f"   Distance: {distance:.4f}")
    print()


Top 3 most similar articles:
1. Article ID: R118
   Similarity Score: -33.2936
   Distance: 33.2936

2. Article ID: R018
   Similarity Score: -35.5547
   Distance: 35.5547

3. Article ID: R025
   Similarity Score: -35.8966
   Distance: 35.8966

